# Better then Bing

In [ ]:
!pip install beir pyserini tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from beir import util
import logging
import pathlib, os
import json

/usr/local/lib/python3.7/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Create json docs

In [ ]:
#### Download scifact.zip dataset and unzip the dataset
dataset = "fiqa"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = "."
data_path = util.download_and_unzip(url, out_dir)

./fiqa.zip:   0%|          | 0.00/17.1M [00:00<?, ?iB/s]

In [ ]:
data = []
with open("fiqa/corpus.jsonl", "r") as file:
    json_list = list(file)

for json_str in json_list:
    res = json.loads(json_str)
    data.append({"id":res.get("_id"), "contents": res.get("text")})

with open("corpus_pyserini.json", "w") as out:
    for entry in data:
        out.write(json.dumps(entry))
        out.write("\n")

## Embed

In [ ]:
!python -m pyserini.encode input --corpus corpus_pyserini.json --fields text  --delimiter "\n" --shard-id 0 --shard-num 1  output --embeddings outs encoder --device cuda:0 --encoder microsoft/xtremedistil-l6-h256-uncased --encoder-class sentence-transformers --fields text --batch 32 --max-length 256 --fp16

57638it [00:00, 138929.70it/s]
 10% 188/1802 [00:12<01:43, 15.62it/s]
Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/pyserini/encode/__main__.py", line 132, in <module>
    embeddings = encoder.encode(**kwargs)
  File "/usr/local/lib/python3.7/dist-packages/pyserini/encode/_auto.py", line 61, in encode
    embeddings = self._mean_pooling(outputs[0], inputs['attention_mask']).detach().cpu().numpy()
KeyboardInterrupt


### Index

In [ ]:
!python -m pyserini.index.faiss --dim 256 --input /content/embedings --output fiqa_index --hnsw

100% 1/1 [00:00<00:00,  1.74it/s]
(6016, 256)
hnsw_add_vertices: adding 6016 elements on top of 0 (preset_levels=0)
  max_level = 1
Adding 13 elements at level 1
Adding 6003 elements at level 0
Done in 4963.093 ms
6016


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Search

In [ ]:
from pyserini.search.faiss import AutoQueryEncoder, FaissSearcher
encoder = AutoQueryEncoder('microsoft/xtremedistil-l6-h256-uncased')

searcher = FaissSearcher(index_dir="fiqa_index", query_encoder=encoder, )

In [ ]:
hits = searcher.search('what is a lobster roll')

In [ ]:
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:7} {hits[i].score:.5f}')

 1 4612    14.43486
 2 52757   14.43390
 3 54004   14.42214
 4 36284   14.38360
 5 19613   14.35964
 6 39976   14.35688
 7 31863   14.34086
 8 2975    14.33475
 9 17348   14.33134
10 14598   14.30120


In [ ]:
hits[0]

DenseSearchResult(docid='4612', score=14.434855)

In [ ]:
from pyserini.search.lucene import LuceneSearcher

JavaException: ignored

In [ ]:
searcher.doc("3817")

AttributeError: ignored